In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [34]:
# #
df = pd.read_csv('/content/data_price_n.csv')
df.head()

,Floor,Floor2,Bathroom,Carpet Area,Price (in rupees)
0,10,11,1,500,6000
1,3,22,2,473,13799
2,10,29,2,779,17500
3,1,3,1,530,6034
4,20,42,2,635,18824


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187531 entries, 0 to 187530
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   Floor              187531 non-null  int64
 1   Floor2             187531 non-null  int64
 2   Bathroom           187531 non-null  int64
 3   Carpet Area        187531 non-null  int64
 4   Price (in rupees)  187531 non-null  int64
dtypes: int64(5)
memory usage: 7.2 MB


# train

In [36]:

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [37]:

# Chia dữ liệu thành features (X) và target (y)
X = df[['Floor', 'Floor2', 'Bathroom', 'Carpet Area']].values
y = df['Price (in rupees)'].values

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Chuyển dữ liệu thành tensor
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test).view(-1, 1)


In [38]:

# Xây dựng mô hình
class HousePriceModel(nn.Module):
    def __init__(self):
        super(HousePriceModel, self).__init__()
        self.fc1 = nn.Linear(4, 64)  # 4 đầu vào
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)   # 1 đầu ra

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [39]:

# Khởi tạo mô hình, loss function và optimizer
model = HousePriceModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Huấn luyện mô hình
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass và tối ưu hóa
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [100/1000], Loss: 781940416.0000
Epoch [200/1000], Loss: 780848256.0000
Epoch [300/1000], Loss: 777692800.0000
Epoch [400/1000], Loss: 771713600.0000
Epoch [500/1000], Loss: 763066432.0000
Epoch [600/1000], Loss: 752933184.0000
Epoch [700/1000], Loss: 743125248.0000
Epoch [800/1000], Loss: 735380224.0000
Epoch [900/1000], Loss: 730317888.0000
Epoch [1000/1000], Loss: 727407872.0000


In [40]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Dự đoán và đánh giá
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor)

    # Hiển thị Test Loss
    print(f'Test Loss: {test_loss.item():.4f}')

    # Hiển thị dự đoán và giá thực tế
    print("Dự đoán giá:", predictions.numpy().flatten())
    print("Giá thực tế:", y_test_tensor.numpy().flatten())

    # Tính toán các chỉ số đánh giá
    mae = mean_absolute_error(y_test_tensor.numpy(), predictions.numpy())
    mse = mean_squared_error(y_test_tensor.numpy(), predictions.numpy())
    r2 = r2_score(y_test_tensor.numpy(), predictions.numpy())

    # Hiển thị các chỉ số đánh giá
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Mean Squared Error (MSE): {mse:.2f}')
    print(f'R-squared (R²): {r2:.4f}')

Test Loss: 456271776.0000
Dự đoán giá: [5352.98   4405.682  7623.5522 ... 5266.281  5261.434  6424.4604]
Giá thực tế: [9646. 4062. 4900. ... 6034. 6182. 7666.]
Mean Absolute Error (MAE): 3007.98
Mean Squared Error (MSE): 456271712.00
R-squared (R²): -0.0015


In [41]:
# Nhập thông số của ngôi nhà mới
new_house = [[2, 2, 2, 500]]  # Ví dụ: Floor=2, Floor2=2, Bathroom=2, Carpet Area=500

# Chuẩn hóa dữ liệu mới
new_house_scaled = scaler.transform(new_house)

# Chuyển dữ liệu mới thành tensor
new_house_tensor = torch.FloatTensor(new_house_scaled)

# Dự đoán giá
model.eval()
with torch.no_grad():
    predicted_price = model(new_house_tensor)

# Hiển thị kết quả
print(f'Dự đoán giá nhà: {predicted_price.item():.2f} rupees')

Dự đoán giá nhà: 6035.20 rupees
